In [ ]:
#Imports
from Options import *
from Convergence import *
from HestonSensitivity import *

In [ ]:
#Plot format
plt.style.use('seaborn-v0_8-bright')
plt.style.use('seaborn-v0_8-darkgrid')

# Part I: Option Valuation
---

### Black Scholes model

* Pricing European put option using Monte Carlo Method

In [ ]:
put = EUPut(
            S0=100,
            K=99,
            T=1,
            r=0.06,
            sigma=0.2,
            simulations=1_000_000,
            time_steps=250,
        )
price = put.price_option()
print(f"Estimated price of put option: {price}")

In [ ]:
Convergence.value_option_black_scholes(
    S_t=100,
    K=99,
    tau=1,
    r=0.06,
    sigma=0.2
)

In [ ]:
put.show_stock_paths()

* Convergence to Black Scholes solution

In [ ]:
Convergence.convergence_to_black_scholes()

* Varying strike price

In [ ]:
Convergence.strike_sensitivity()

* Varying sigma 

In [ ]:
Convergence.sigma_sensitivity()

* Standard error of estimate

In [ ]:
Convergence.standard_error()

### Heston model

* Pricing barrier option implemented with the Heson model and the Milstein scheme

In [ ]:
upAndOutCall = EuUpAndOutCall(
            S0=100,
            K=100,
            T=1,
            r=0.06,
            sigma=0.2,
            simulations=10_000,
            time_steps=250,
            barrier = 120, 
            theta = 0.04,
            kappa = 2, 
            epsilon = 0.1,
            rho =  -0.7,
            V0 = 0.10 
        ) 
price = upAndOutCall.price_option()
print(f"Estimated price of put option: {price}")

* Sensitivity to barrier level

In [ ]:
sensitivityAnalysis.barrier_sensitivity()

* Sensitivity to correlation ($p$)

In [ ]:
sensitivityAnalysis.correlation_sensitivity()

* Sensitivity to vol-of-vol ($\epsilon$)

In [ ]:
sensitivityAnalysis.vol_of_vol_sensitivity()